<a href="https://colab.research.google.com/github/EmidioLP/Freecodecamp/blob/main/Notebooks/Machine%20Learning%20with%20Python/Classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse notebook é baseado nas aulas disponibilizadas em: https://www.freecodecamp.org/portuguese/learn/machine-learning-with-python/

# Classificação
 
 Enquanto a regressão é usada para prever um valor numérico, a classificação é usada para separar pontos de dados em classes de diferentes labels. 

 Esse notebook é baseado no guia disponibilizado no site do Tensorflow. Para mais informações, acessar o link a seguir: https://www.tensorflow.org/tutorials/estimator/premade

# Imports e Setup do sistema

In [ ]:
tensorflow_version 2.x #Apenas é necessária caso esteja trabalhando com um notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf

# Base de dados

A base de dados que iremos utilizar separa tipos de flores em 3 diferentes espécies. 


*   Setosa
*   Versicolor
*   Virginica

As informações sobre cada flor são as seguintes:


*   sepal lenght (comprimento da sépala)
*   sepal width  (largura da sépala)
*   petal length (comprimento da pétala)
*   petal width  (largura da pétala)

In [ ]:
# Definição de algumas colunas constantes para utilização futura
CSV_COLUMN_NAMES = ['SepalLenght', 'SepalWidth', 'PetalLenght', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Aqui iremos utilizar o keras (um módulo do Tensorflow) para pegar nossas bases de dados e lê-las dentro de um dataframe pandas

In [ ]:
# É importante sempre verificar os dados
train.head()

,SepalLenght,SepalWidth,PetalLenght,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y = train.pop("Species")
test_y = test.pop("Species")
train.head() #Verificar se a coluna "species" foi retirada

,SepalLenght,SepalWidth,PetalLenght,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape #Vamos verificar o tamanho do nosso dataframe

(120, 4)

# Input Function

Assim como no modulo anterior, é necessária a criação de uma função para modificar como os dados serão inseridos no modelo, pelo fato da necessidade de transformação do dataframe para um **tf.data.Dataset**.

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
  # Converte os inputs para um Dataset.
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  
  # Embaralha e repete o processo se estiver em modo de treino
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

# Feature Columns

As features columns definem como usaremos nosso input!

In [ ]:
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLenght', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLenght', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Criando o modelo

Para tarefas de classificação existem uma variedade de modelos/estimadores para escolhermos. Algumas opções a seguir:


*   **DNNClassifier** (Deep Neural Network)
*   **LinearClassifier**

Pode-se escolher qualquer modelo, mas o DNN é a escolha recomendada pelo Tensorflow. Isso porquê nós talvez não conseguissemos achar uma correspondência linear em nossos dados.

In [ ]:
# Criando um modelo DNN com 2 camadas ocultas com 30 e 10 nodes cada.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #Duas camadas ocultas de 30 e 10 nodes, respectivamente.
    hidden_units=[30,10],
    # O modelo deve escolher entre 3 classes.
    n_classes=3)

# Treinamento

Após criado, é hora de treinarmos nosso modelo.

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# Incluimos lambda para evitar a criação de uma função interior previamente.

In [ ]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test accuracy: 0.900



#Predições

Agora que já treinamos os modelos, temos que verificar quais serão as predições dele.

In [ ]:
def input_fn(features, batch_size=256):
  #Converte as entradas para um Dataset sem labels.
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLenght', 'SepalWidth', 'PetalLenght', 'PetalWidth']
predict = {}

print("Escreva o valor numerico:")
for feature in features:
  valid=True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

Escreva o valor numerico:
SepalLenght: 2.4
SepalWidth: 2.6
PetalLenght: 6.5
PetalWidth: 6.3
Prediction is "Virginica" (98.6%)
